# Analyzing CIA Factbook Data Using SQL

In this project, we will be looking at demographic data for various countries from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all the countries on Earth.The Factbook contains demographic information like the following:

* population — the global population.
* population_growth — the annual population growth rate, as a percentage.
* area — the total land and water area.

## Load and Select Table

In [23]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [24]:
%%sql

SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Explore Table

In [25]:
%%sql
SELECT *
    FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* name — the name of the country.
* area— the country's total area (both land and water).
* area_land — the country's land area in square kilometers.
* area_water — the country's waterarea in square kilometers.
* population — the country's population.
* population_growth— the country's population growth as a percentage.
* birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
* death_rate — the country's death rate, or the number of death per year per 1,000 people.
* Let's start by calculating some summary statistics and look for any outlier countries.

In [26]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In [27]:
%%sql

SELECT name, population
    FROM facts
 WHERE population = (SELECT MIN(population) FROM facts);

Done.


name,population
Antarctica,0


In [28]:
%%sql

SELECT name, population
    FROM facts
 WHERE population = (SELECT MAX(population) FROM facts);

Done.


name,population
World,7256490011


In [29]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
 WHERE name != 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [30]:
%%sql

SELECT AVG(population), AVG(area)
    FROM facts
 WHERE name <> 'World';

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


## High Population & Low Area Countries

In [31]:
%%sql

SELECT name, population,area, population/area AS ratio
    FROM facts
 WHERE population > (SELECT AVG(population) FROM facts)
 AND area < (SELECT AVG(area) FROM facts)
 ORDER BY ratio DESC;

Done.


name,population,area,ratio
Bangladesh,168957745,148460,1138
Philippines,100998376,300000,336
Japan,126919659,377915,335
Vietnam,94348835,331210,284
United Kingdom,64088222,243610,263
Germany,80854408,357022,226
Thailand,67976405,513120,132


## Highest Population & Growth Rate

In [32]:
%%sql

SELECT name, MAX(population)
    FROM facts
 WHERE name <> 'World';

Done.


name,MAX(population)
China,1367485388


In [33]:
%%sql

SELECT name, MAX(population_growth)
    FROM facts
 WHERE name <> 'World';

Done.


name,MAX(population_growth)
South Sudan,4.02


## Water-Land Ratio

In [34]:
%%sql

SELECT name, area, area_water, area_land, ROUND((CAST(area_water AS FLOAT) / area*100),2) || '%' AS 'water to land ratio'
    FROM facts
 ORDER BY ROUND((CAST(area_water AS FLOAT) / area*100),2) DESC
LIMIT 20;



Done.


name,area,area_water,area_land,water to land ratio
British Indian Ocean Territory,54400,54340,60,99.89%
Virgin Islands,1910,1564,346,81.88%
Puerto Rico,13791,4921,8870,35.68%
"Bahamas, The",13880,3870,10010,27.88%
Guinea-Bissau,36125,8005,28120,22.16%
Malawi,118484,24404,94080,20.6%
Netherlands,41543,7650,33893,18.41%
Uganda,241038,43938,197100,18.23%
Eritrea,117600,16600,101000,14.12%
Liberia,111369,15049,96320,13.51%


In [35]:
%%sql

SELECT id, name, area, area_water, area_land, population, population_growth
    FROM facts
 WHERE area_water > area_land;




Done.


id,name,area,area_water,area_land,population,population_growth
228,British Indian Ocean Territory,54400,54340,60,None,None
247,Virgin Islands,1910,1564,346,103574,0.59


## Highest Population Increase

In [51]:
%%sql

SELECT id,name,population,population_growth AS 'growth_rate (%)',CAST((population*(population_growth/100)) AS INT) `net_increase`
    FROM facts
 WHERE name <> 'World'   
 ORDER BY `net_increase` DESC
 LIMIT 20;

 

Done.


id,name,population,growth_rate (%),net_increase
77,India,1251695584,1.22,15270686
37,China,1367485388,0.45,6153684
129,Nigeria,181562056,2.45,4448270
132,Pakistan,199085847,1.46,2906653
58,Ethiopia,99465819,2.89,2874562
14,Bangladesh,168957745,1.6,2703323
186,United States,321368864,0.78,2506677
78,Indonesia,255993674,0.92,2355141
40,"Congo, Democratic Republic of the",79375136,2.45,1944690
138,Philippines,100998376,1.61,1626073


## Death Rate & Birth Rate

In [37]:
%%sql

SELECT id,name,death_rate,birth_rate, ROUND(death_rate/birth_rate,2) 'death-birth ratio'
    FROM facts
 WHERE death_rate > birth_rate AND (death_rate/birth_rate) > 1
 ORDER BY ROUND(death_rate/birth_rate,2) DESC
LIMIT 20;

Done.


id,name,death_rate,birth_rate,death-birth ratio
26,Bulgaria,14.44,8.92,1.62
153,Serbia,13.66,9.08,1.5
96,Latvia,14.31,10.0,1.43
102,Lithuania,14.27,10.1,1.41
75,Hungary,12.73,9.16,1.39
117,Monaco,9.24,6.65,1.39
65,Germany,11.42,8.47,1.35
158,Slovenia,11.37,8.42,1.35
183,Ukraine,14.46,10.72,1.35
214,Saint Pierre and Miquelon,9.72,7.42,1.31


## Population/Area Ratio

In [38]:
%%sql

SELECT name,population,area,population/area AS ratio
    FROM facts
    ORDER BY ratio DESC
    LIMIT 10;

Done.


name,population,area,ratio
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


The list of countries with the highest population/area ratio are islands, autonomous regions, and generally small areas that have a large population in proportion. Comparing to the query selecting for above average populations and below average area, the list of countries are very different. This is mainly because the high population to area ratio countries are smaller nations/islands, wheras the countries listed in the other query are larger countries (therefore smaller ratio). 

# Conclusion

From this analysis, we can conclude that:

* Above-average population and low-area countries have a lower population-area ratio than smaller countries/city states/islands which can have much larger population in proportion to the area 

* China has the highest population (~ 1.37 billion), while South Sudan has the highest population growth rate (4.02%) 

* Islands have the highest water to land ratio

* India, China, and Nigeria have the highest population increase

* A lot of the countries with a high death-birth ratio tend to be Balkan/Southeast European countries (The reason for this is something that can be further explored)